In [ ]:
from google.colab import drive
drive.mount('/content/drive')

general_path = "/content/drive/My Drive/Colab Notebooks/lafin"
%cd /content/drive/My\ Drive/Colab Notebooks/lafin

In [3]:
!pip install -r "/content/drive/My Drive/Colab Notebooks/lafin/requirements.txt"
!pip install scipy==1.2.2
!pip install face-alignment

## Celeba Resize

In [ ]:
import os 

from PIL import Image, ImageOps
from pathlib import Path

# Function that resizes images to the required LAFIN sizes
def resize_images(occ_dir, res_dir, new_size_x, new_size_y):
  occ_list = os.listdir(occ_dir)

  for occ in occ_list:
    occ_path = os.path.join(occ_dir, occ)

    face = Image.open(occ_path)
    face = ImageOps.exif_transpose(face) # rotate image if necessary
    face = face.resize((new_size_x, new_size_y), Image.ANTIALIAS) # apply the new sizes for the original image 
    
    new_occ_path = os.path.join(res_dir, occ)
    face.save(new_occ_path) 

***Store the occluded images  under `lafin/data/images/` path and the occlusion masks under `lafin/data/masks/` path. <br> Samples of occluded images and masks are provided.***

In [ ]:
occ_dir = os.path.join(general_path, "data/images/")
mask_dir = os.path.join(general_path, "data/masks/")

# Resize to LAFIN required sizes
resize_images(occ_dir, occ_dir, 256, 256)
resize_images(mask_dir, mask_dir, 512, 512)

## FAN landmarks

In [ ]:
from fan_func import fan_func

occ_dir = os.path.join(general_path, "data/images/")

# Get the FAN landmarks and store them in txt files
not_detected = fan_func(image_path=occ_dir, landmark_path="./data/landmarks/", fileList_path="./myceleba_landmarks.flist")

## Lafin

In [ ]:
from lafin_func import lafin_func

# Images with FAN landmarks
lafin_func(model_type=2, config_path='./checkpoints/celeba/config.yml')

***Create `inpainting_steps/` directory inside `lafin/` directory and insert the initial images of `lafin/data/images` directory in a new `inpainting_steps/occlusions/` directory. At the same time, create `inpainting_steps/result/`and `inpainting_steps/landmark_mask/` directories and insert in them the corresponding images, as retrieved from the `lafin/checkpoints/celeba/results/inpaint/` path.***

## PNG to JPG

In [ ]:
import os
from PIL import Image

# Function that converts png images to jpg images
def png_to_jpg(data_dir):
  for subdir, dirs, files in os.walk(data_dir):  
    for file in files:
      path = os.path.join(subdir, file) 
      im = Image.open(path)      

      name, ext = os.path.splitext(file)
      file_name = name + '.jpg'
      new_path = os.path.join(subdir, file_name) 
      im.save(new_path)


# Covert images to their initial format
lafin_path = "./inpainting_steps/"

lafin_dirs = os.listdir(lafin_path)

for dir in lafin_dirs:
  l_path = os.path.join(lafin_path, dir)
  png_to_jpg(l_path)

In [ ]:
# Function that removes png images
def remove_png(data_dir):
  for subdir, dirs, files in os.walk(data_dir):  
    for file in files:
      name, ext = os.path.splitext(file)
      
      if ext == '.png':
        path = os.path.join(subdir, file) 
        os.remove(path)

# Remove the png images and keep only the jpg ones
for dir in lafin_dirs:
  l_path = os.path.join(lafin_path, dir)
  remove_png(l_path)

## Resize to RPCA sizes

In [ ]:
import os

# Resize images to match the RPCA sizes for evaluation purposes

lafin_dirs = ["./inpainting_steps/"] # list many inpainting_steps/ directories of all occlusion sizes

for ldir in lafin_dirs:
  dirs_list = os.listdir(ldir)

  for subdir in dirs_list:
    subdir_path = os.path.join(ldir, subdir)  
    resize_images(subdir_path, subdir_path, 100, 102)

## 1 Inpainting per Identity

In [ ]:
import os
from PIL import Image

# Function that creates a directory if it doesn't already exist 
def create_dir(dir):
  if not os.path.exists(dir):
    os.makedirs(dir)

# Function that groups the inpainted images by identity
def inp_per_id(example_dir, data_dir, root_dir, full=True):
  occ_list = os.listdir(os.path.join(data_dir, "result"))
  
  for subdir, dirs, files in os.walk(example_dir):
    for dir in dirs:
      id_dir = os.path.join(root_dir, dir)
      create_dir(id_dir)

    for file in files:
      if file in occ_list:      
        example_path = os.path.join(subdir, file)
        
        data_dirs = os.listdir(data_dir)
      
        for d in data_dirs:
          data_path = os.path.join(data_dir, d)
          file_path = os.path.join(data_path, file)
          try:    
            im = Image.open(file_path)
          except OSError as e:
            continue

          id_name = os.path.basename(subdir)
          id_path = os.path.join(root_dir, id_name)

          name, ext = os.path.splitext(file)

          if full:
            if d == 'occlusions':
              file_name = name + '_X' + ext
            elif d == 'result':
              file_name = name + '_inp' + ext
            else:  
              file_name = name + '_lm' + ext
          else:
            if d == 'result':
              file_name = name + '_inp' + ext
            else:
              continue
              
          grouped_path = os.path.join(id_path, file_name)
          im.save(grouped_path)

In [ ]:
example_dir = "../datasets/small/myceleba_split/test"
data_dir = "./inpainting_steps"
root_dir = "../results/myceleba_lafin"
steps_dir = "../results/myceleba_lafin/steps"
inp_dir = "../results/myceleba_lafin/inp"
create_dir(root_dir)
create_dir(steps_dir)
create_dir(inp_dir)

# Group inpainted images by identity
inp_per_id(example_dir, data_dir, steps_dir)
inp_per_id(example_dir, data_dir, inp_dir, False)